# Calibration & Validation Framework - Interactive Debugging

This notebook helps **tune anatomical ROIs** and **validate threshold choices** for the sinus CT pipeline.

## Goals
1. Visualize HU calibration anchors (air, bone)
2. Inspect adaptive threshold histograms
3. Debug OMC corridor placement (why 0% air?)
4. Verify wall shell and sclerosis detection
5. Iterate on heuristics until Orlando normal scan passes


In [ ]:
import sys
from pathlib import Path
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import json

# Add src to path
sys.path.append(str(Path('src')))

from calibration import (
    calibrate_volume, 
    adaptive_threshold_air_tissue, 
    detect_air_anchor, 
    detect_bone_anchor
)
from sinus import (
    build_sinus_wall_shell,
    compute_sclerosis_zscore,
    estimate_reference_bone_stats,
    measure_omc_patency_coronal,
    detect_retention_cysts_strict,
)

print("✓ Modules loaded")

## 1. Load and Calibrate Volume

In [ ]:
# Load raw volume
nifti_path = Path('data/processed/sinus_ct.nii.gz')
img = nib.load(str(nifti_path))
volume_raw = img.get_fdata().astype(np.float32)
spacing = img.header.get_zooms()[:3]

print(f"Shape: {volume_raw.shape}")
print(f"Spacing: {spacing} mm")
print(f"HU range (raw): [{volume_raw.min():.0f}, {volume_raw.max():.0f}]")

# Calibrate
volume, cal_meta = calibrate_volume(volume_raw)
print(f"\nHU range (calibrated): [{volume.min():.0f}, {volume.max():.0f}]")
print(f"Correction applied: {cal_meta['applied']}")
if cal_meta['correction']:
    print(f"  Slope: {cal_meta['correction']['slope']:.4f}")
    print(f"  Intercept: {cal_meta['correction']['intercept']:.2f}")

## 2. Visualize HU Calibration Anchors

In [ ]:
# Show air and bone anchor regions
z, y, x = volume.shape

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Air anchor: peripheral slices
axes[0].imshow(volume[5, :, :], cmap='gray', vmin=-1000, vmax=400)
axes[0].set_title(f"Air Anchor Region (peripheral)\nMeasured: {cal_meta['air_anchor']['measured_hu']:.1f} HU")
axes[0].axis('off')

# Bone anchor: inferior-central (hard palate)
z_palate = int(z * 0.7)
axes[1].imshow(volume[z_palate, :, :], cmap='gray', vmin=-1000, vmax=1500)
y_c, x_c = y//2, x//2
y_m, x_m = int(y * 0.2), int(x * 0.2)
rect = Rectangle((x_c - x_m, y_c - y_m), 2*x_m, 2*y_m, linewidth=2, edgecolor='cyan', facecolor='none')
axes[1].add_patch(rect)
axes[1].set_title(f"Bone Anchor Region (hard palate)\nMeasured: {cal_meta['bone_anchor']['measured_hu']:.1f} HU")
axes[1].axis('off')

# Histogram of anchors
air_roi_flat = volume[:10, :, :].ravel()
air_vox = air_roi_flat[air_roi_flat < -800]
bone_roi = volume[int(z*0.6):int(z*0.8), y_c-y_m:y_c+y_m, x_c-x_m:x_c+x_m]
bone_vox = bone_roi[(bone_roi > 800) & (bone_roi < 1400)]

axes[2].hist(air_vox, bins=50, alpha=0.6, label=f'Air (n={len(air_vox)})', color='blue')
axes[2].hist(bone_vox, bins=50, alpha=0.6, label=f'Bone (n={len(bone_vox)})', color='orange')
axes[2].axvline(cal_meta['air_anchor']['expected_hu'], color='blue', linestyle='--', label='Expected air')
axes[2].axvline(cal_meta['bone_anchor']['expected_hu'], color='orange', linestyle='--', label='Expected bone')
axes[2].set_xlabel('HU')
axes[2].set_ylabel('Count')
axes[2].legend()
axes[2].set_title('Anchor HU Distributions')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('docs/validation/calibration_anchors.png', dpi=120, bbox_inches='tight')
plt.show()

print("✓ Saved: docs/validation/calibration_anchors.png")

## 3. Adaptive Thresholds from Histogram

In [ ]:
# Build rough cavity mask
air_mask_rough = volume < -400
from scipy import ndimage
air_mask_clean = ndimage.binary_opening(air_mask_rough, structure=np.ones((3,3,3)))

thresholds = adaptive_threshold_air_tissue(volume, sinus_mask=air_mask_clean)

# Plot histogram with detected thresholds
cavity_hu = volume[air_mask_clean].ravel()

fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(cavity_hu, bins=140, range=(-1000, 400), alpha=0.7, color='steelblue', edgecolor='black')
ax.axvline(thresholds['air_peak'], color='blue', linestyle='--', linewidth=2, label=f"Air peak: {thresholds['air_peak']:.0f} HU")
ax.axvline(thresholds['tissue_peak'], color='red', linestyle='--', linewidth=2, label=f"Tissue peak: {thresholds['tissue_peak']:.0f} HU")
ax.axvline(thresholds['air_threshold'], color='green', linestyle='-', linewidth=2, label=f"Air threshold: {thresholds['air_threshold']:.0f} HU")
ax.set_xlabel('HU', fontsize=12)
ax.set_ylabel('Voxel Count', fontsize=12)
ax.set_title('Adaptive Threshold Detection (Cavity Histogram)', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('docs/validation/adaptive_thresholds.png', dpi=120, bbox_inches='tight')
plt.show()

print(f"✓ Air threshold: {thresholds['air_threshold']:.1f} HU")
print(f"  Saved: docs/validation/adaptive_thresholds.png")

## 4. Debug OMC Corridor Placement ⚠️

**Why 0% air?** Let's visualize where the corridor ROI is landing.

In [ ]:
# Extract OMC corridor coordinates
z, y, x = volume.shape
midline = x // 2

z_start = int(z * 0.40)
z_end = int(z * 0.60)
y_start = int(y * 0.55)
y_end = int(y * 0.75)

x_left = slice(midline - 35, midline - 5)
x_right = slice(midline + 5, midline + 35)

# Show corridor on representative slices
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Coronal view (z slices) - where OMC should be
slice_indices = [z_start, (z_start + z_end)//2, z_end - 1]
for i, z_idx in enumerate(slice_indices):
    axes[0, i].imshow(volume[z_idx, :, :], cmap='gray', vmin=-1000, vmax=400)
    
    # Draw corridor boxes
    rect_left = Rectangle((midline - 35, y_start), 30, y_end - y_start, linewidth=2, edgecolor='cyan', facecolor='none', label='Left OMC')
    rect_right = Rectangle((midline + 5, y_start), 30, y_end - y_start, linewidth=2, edgecolor='yellow', facecolor='none', label='Right OMC')
    axes[0, i].add_patch(rect_left)
    axes[0, i].add_patch(rect_right)
    axes[0, i].axvline(midline, color='red', linestyle='--', linewidth=1, alpha=0.5)
    axes[0, i].set_title(f'Coronal slice z={z_idx} ({z_idx/z*100:.0f}%)')
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].legend(loc='upper left', fontsize=8)

# Axial view (y slices) - check anterior-posterior placement
y_indices = [y_start, (y_start + y_end)//2, y_end - 1]
for i, y_idx in enumerate(y_indices):
    axes[1, i].imshow(volume[:, y_idx, :], cmap='gray', vmin=-1000, vmax=400, aspect='auto')
    axes[1, i].axvline(midline, color='red', linestyle='--', linewidth=1, alpha=0.5)
    axes[1, i].axhline(z_start, color='cyan', linestyle='--', linewidth=1, alpha=0.5)
    axes[1, i].axhline(z_end, color='cyan', linestyle='--', linewidth=1, alpha=0.5)
    axes[1, i].set_title(f'Axial slice y={y_idx} ({y_idx/y*100:.0f}%)')
    axes[1, i].axis('off')

plt.tight_layout()
plt.savefig('docs/validation/omc_corridor_debug.png', dpi=120, bbox_inches='tight')
plt.show()

# Compute air fraction in corridor
left_corridor = volume[z_start:z_end, y_start:y_end, x_left]
right_corridor = volume[z_start:z_end, y_start:y_end, x_right]

left_air = (left_corridor < thresholds['air_threshold']).sum() / left_corridor.size
right_air = (right_corridor < thresholds['air_threshold']).sum() / right_corridor.size

print(f"✓ Left corridor air fraction: {left_air*100:.2f}%")
print(f"✓ Right corridor air fraction: {right_air*100:.2f}%")
print(f"  Saved: docs/validation/omc_corridor_debug.png")
print("\n⚠️  If both are ~0%, the corridor is hitting bone/tissue instead of the infundibulum.")
print("   → Adjust z_start/z_end or y_start/y_end to move anteriorly/posteriorly")

## 5. Sinus Wall Shell Visualization

In [ ]:
# Build wall shell
wall_shell = build_sinus_wall_shell(air_mask_clean, shell_thickness=2)

# Show on mid-coronal slice
z_mid = z // 2

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(volume[z_mid, :, :], cmap='gray', vmin=-1000, vmax=400)
axes[0].set_title('Original (Coronal Mid)')
axes[0].axis('off')

axes[1].imshow(volume[z_mid, :, :], cmap='gray', vmin=-1000, vmax=400)
axes[1].imshow(air_mask_clean[z_mid, :, :], cmap='Blues', alpha=0.3)
axes[1].set_title('Air Cavities (blue)')
axes[1].axis('off')

axes[2].imshow(volume[z_mid, :, :], cmap='gray', vmin=-1000, vmax=400)
axes[2].imshow(wall_shell[z_mid, :, :], cmap='Reds', alpha=0.5)
axes[2].set_title('Wall Shell (red, 2 voxel thick)')
axes[2].axis('off')

plt.tight_layout()
plt.savefig('docs/validation/wall_shell_vis.png', dpi=120, bbox_inches='tight')
plt.show()

print(f"✓ Wall shell volume: {wall_shell.sum()} voxels")
print(f"  Saved: docs/validation/wall_shell_vis.png")

## 6. Sclerosis Z-Score Distribution

In [ ]:
# Compute reference bone and sclerosis
ref_bone = estimate_reference_bone_stats(volume)
sclerosis = compute_sclerosis_zscore(
    volume,
    wall_shell,
    reference_bone_hu=ref_bone,
    z_threshold=2.0,
    min_cluster_size=30,
)

# Histogram of wall shell HU
wall_hu = volume[wall_shell > 0]

fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(wall_hu, bins=60, alpha=0.7, color='coral', edgecolor='black')
ax.axvline(ref_bone[0], color='blue', linestyle='--', linewidth=2, label=f'Ref bone mean: {ref_bone[0]:.0f} HU')
ax.axvline(ref_bone[0] + 2*ref_bone[1], color='red', linestyle='--', linewidth=2, label=f'Sclerosis (z>2): {sclerosis["threshold_hu"]:.0f} HU')
ax.set_xlabel('HU', fontsize=12)
ax.set_ylabel('Voxel Count', fontsize=12)
ax.set_title(f'Sinus Wall HU Distribution (Sclerotic: {sclerosis["sclerotic_fraction"]*100:.2f}%)', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('docs/validation/sclerosis_histogram.png', dpi=120, bbox_inches='tight')
plt.show()

print(f"✓ Reference bone: {ref_bone[0]:.0f} ± {ref_bone[1]:.0f} HU")
print(f"✓ Sclerotic fraction: {sclerosis['sclerotic_fraction']*100:.2f}% (target < 5%)")
print(f"  Saved: docs/validation/sclerosis_histogram.png")

## 7. Next Steps

**OMC Corridor Tuning:**
- The current corridor may be too posterior or too superior
- Try adjusting:
  - `z_start/z_end`: move more superior (decrease both) or inferior (increase both)
  - `y_start/y_end`: move more anterior (decrease both) or posterior (increase both)
- Ideal: corridor should pass through the infundibulum where air connects maxillary sinus to middle meatus

**Validation Loop:**
1. Adjust heuristics in `src/anatomical_metrics.py`
2. Re-run `tests/test_orlando_normal.py`
3. Iterate until OMC shows Patent bilaterally

**Add More Test Cases:**
- One truly inflamed scan (known mucosal thickening)
- One mild allergy day (trace findings)
- One with confirmed cyst
- Build a multi-case validation suite